In [1]:
import pandas as pd
import utils
import plotly.graph_objects as go
import instrument

In [2]:
pair = "EUR_USD"
granularity = "H1"
ma_list = [10,16,64]
i_pair = instrument.Instrument.get_instrument_by_name("EUR_USD")

In [3]:
df = pd.read_pickle(utils.get_his_data_filename(pair, granularity))
non_cols = ['time', 'volume']
mod_cols = [x for x in df.columns if x not in non_cols]
df[mod_cols] = df[mod_cols].apply(pd.to_numeric)

In [4]:
df_ma = df[['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c']].copy()
for ma in ma_list:  
    df_ma[f'MA_{ma}'] = df_ma.mid_c.rolling(window=ma).mean()
df_ma.dropna(inplace=True)
df_ma.reset_index(drop=True, inplace=True)

In [5]:
df_ma.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_10,MA_16,MA_64
0,2023-01-04 14:00:00+00:00,1.06106,1.06285,1.06023,1.06246,1.060275,1.058783,1.062421
1,2023-01-04 15:00:00+00:00,1.06244,1.06253,1.05814,1.06032,1.060613,1.059143,1.062261
2,2023-01-04 16:00:00+00:00,1.06032,1.06208,1.05927,1.06189,1.061102,1.059522,1.062123
3,2023-01-04 17:00:00+00:00,1.06193,1.06201,1.06048,1.06074,1.061238,1.059776,1.061981
4,2023-01-04 18:00:00+00:00,1.06074,1.06129,1.05956,1.05999,1.061080,1.059994,1.061832


In [6]:
def is_trade(row):
    if row.DIFF >= 0 and row.DIFF_PREV < 0:
        return 1
    if row.DIFF <= 0 and row.DIFF_PREV > 0:
        return -1
    return 0

In [7]:
df_ma['DIFF'] = df_ma.MA_16 - df_ma.MA_64
df_ma['DIFF_PREV'] = df_ma.DIFF.shift(1)
df_ma['IS_TRADE'] = df_ma.apply(is_trade, axis=1)
df_trades = df_ma[df_ma.IS_TRADE!=0].copy()

In [8]:
df_trades.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_10,MA_16,MA_64,DIFF,DIFF_PREV,IS_TRADE
11,2023-01-05 01:00:00+00:00,1.06296,1.06316,1.06185,1.06202,1.061027,1.061086,1.060975,0.000110,-0.000041,1
27,2023-01-05 17:00:00+00:00,1.05212,1.05332,1.05188,1.05300,1.057344,1.058646,1.058926,-0.000279,0.000081,-1
57,2023-01-08 23:00:00+00:00,1.06423,1.06630,1.06420,1.06628,1.062459,1.058194,1.057850,0.000344,-0.000487,1
198,2023-01-16 20:00:00+00:00,1.08194,1.08208,1.08176,1.08206,1.081969,1.082396,1.082419,-0.000023,0.000256,-1
244,2023-01-18 18:00:00+00:00,1.07962,1.07994,1.07870,1.07967,1.083371,1.082021,1.081933,0.000088,-0.000079,1


In [9]:
df_trades["DELTA"] = (df_trades.mid_c.diff()/i_pair.pipLocation).shift(-1)

In [10]:
df_trades["GAIN"] = df_trades.DELTA * df_trades.IS_TRADE

In [11]:
df_trades.GAIN.sum()

-708.3999999999957

In [23]:
df_plot = df_ma.iloc[-1000:].copy()

In [24]:
df_plot.shape

(1000, 11)

In [25]:
fig = go.Figure()
fig.add_trace(go.Candlestick(
    x=df_plot.time, open=df_plot.mid_o, high=df_plot.mid_h, low=df_plot.mid_l, close=df_plot.mid_c,
    line=dict(width=1), opacity=1,
    increasing_fillcolor='#24A06B',
    decreasing_fillcolor="#CC2E3C",
    increasing_line_color='#2EC886',  
    decreasing_line_color='#FF3A4C'
))
for ma in ma_list:  
    col = f"MA_{ma}"
    fig.add_trace(go.Scatter(x=df_plot.time, 
        y=df_plot[col],
        line=dict(width=2),
        line_shape='spline',
        name=col
        ))
fig.update_layout(width=1000,height=400,
    margin=dict(l=10,r=10,b=10,t=10),
    font=dict(size=10,color="#e1e1e1"),
    paper_bgcolor="#1e1e1e",
    plot_bgcolor="#1e1e1e")
fig.update_xaxes(
    gridcolor="#1f292f",
    showgrid=True,fixedrange=True,rangeslider=dict(visible=False)
)
fig.update_yaxes(
    gridcolor="#1f292f",
    showgrid=True
)
fig.show()

In [15]:
all = pd.read_pickle('all_trades.pkl')
all.describe()

,mid_c,DIFF,DIFF_PREV,IS_TRADE,DELTA,GAIN,MASHORT,MALONG,DURATION
count,66262.000000,66262.000000,6.626200e+04,66262.000000,66262.000000,66262.000000,66262.000000,66262.000000,66262.000000
mean,40.572669,-0.000123,7.507401e-05,-0.001388,2.273744,-2.823576,11.762277,60.263077,50.028870
std,66.252171,0.019440,1.931071e-02,1.000007,59.884409,59.861004,12.445540,63.357956,83.821603
min,0.518710,-0.346129,-3.557461e-01,-1.000000,-758.300000,-424.700000,4.000000,8.000000,1.000000
25%,0.870980,-0.000172,-1.685938e-04,-1.000000,-18.400000,-26.400000,4.000000,16.000000,6.000000
50%,1.352580,0.000000,3.125000e-07,-1.000000,1.200000,-8.900000,8.000000,32.000000,17.000000
75%,90.378000,0.000168,1.718750e-04,1.000000,21.400000,6.200000,16.000000,96.000000,61.000000
max,194.750000,0.369523,3.314141e-01,1.000000,720.600000,758.300000,64.000000,256.000000,1613.000000
